In [1]:
!pip install numpy==1.15

    100% |████████████████████████████████| 13.9MB 3.3MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [1]:
!pip install --upgrade pandas folium imgaug

    100% |████████████████████████████████| 10.1MB 4.8MB/s 
Requirement already up-to-date: folium in /usr/local/lib/python3.6/dist-packages (0.8.3)
Requirement already up-to-date: imgaug in /usr/local/lib/python3.6/dist-packages (0.2.8)
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [1]:
!pip3 install https://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

    100% |████████████████████████████████| 591.8MB 26kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [3]:
!pip3 install torchvision

In [2]:
!pip install pytorch-pretrained-bert==v0.6.1

    100% |████████████████████████████████| 122kB 3.8MB/s 


In [4]:
!git clone -b master https://github.com/charles9n/bert-sklearn

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 107, done.
remote: Total 107 (delta 0), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (107/107), 378.81 KiB | 8.42 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [5]:
cd bert-sklearn

/content/bert-sklearn


In [6]:
!pip install .

Processing /content/bert-sklearn
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn


In [7]:
cd ..

/content


In [8]:
import numpy as np
import pandas as pd
import os
import sys
import csv
from sklearn import metrics
from sklearn.metrics import classification_report

sys.path.append("../") 
from bert_sklearn import BertClassifier
from bert_sklearn import load_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [9]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [23]:
!ls

adc.json      dev.tsv	 in_domain_dev.tsv    out_of_domain_dev.tsv  test.tsv
bert-sklearn  drive	 in_domain_train.tsv  raw		     train.tsv
CoLA	      glue_data  original	      sample_data


In [0]:
!mkdir -p glue_data

In [24]:
cd glue_data

/content/glue_data


In [0]:
!mkdir -p CoLA

In [27]:
cd CoLA

/content/glue_data/CoLA


In [28]:
from google.colab import files

uploaded = files.upload()

Saving dev.tsv to dev.tsv
Saving test.tsv to test.tsv
Saving train.tsv to train.tsv


In [0]:
!mkdir -p original

In [30]:
cd original

/content/glue_data/CoLA/original


In [0]:
!mkdir -p raw

In [32]:
cd raw

/content/glue_data/CoLA/original/raw


In [33]:
from google.colab import files

uploaded = files.upload()

Saving in_domain_dev.tsv to in_domain_dev.tsv
Saving in_domain_train.tsv to in_domain_train.tsv
Saving out_of_domain_dev.tsv to out_of_domain_dev.tsv


In [34]:
cd ..

/content/glue_data/CoLA/original


In [0]:
!mkdir -p tokenized

In [36]:
cd tokenized

/content/glue_data/CoLA/original/tokenized


In [44]:
from google.colab import files

uploaded = files.upload()

Saving download_glue_data.py to download_glue_data.py


In [41]:
cd ..

/content


In [0]:
DATADIR = os.getcwd() + '/glue_data'

In [45]:
!python3 download_glue_data.py --data_dir glue_data --tasks CoLA 

	Completed!


In [46]:
"""
CoLA train data size: 8551 
CoLA dev data size: 1043 
"""

def get_cola_df(filename,cols = [3,1]):
    df = pd.read_csv(filename, sep='\t',  encoding = 'utf8',keep_default_na=False,header=None)
    df = df[cols]
    df.columns=['text','label']
    return df

def get_cola_data(train_file = DATADIR+'/CoLA/train.tsv', 
                   dev_file =  DATADIR+'/CoLA/dev.tsv'):
    
    train = get_cola_df(train_file)
    print("CoLA train data size: %d "%(len(train)))
    dev = get_cola_df(dev_file)
    print("CoLA dev data size: %d "%(len(dev)))

    label_list = np.unique(train['label'].values)
    return train,dev,label_list  
                  
train,dev,label_list =  get_cola_data()

CoLA train data size: 8551 
CoLA dev data size: 1043 


In [47]:
print(label_list)

[0 1]


In [0]:
train = train.sample(len(train))

In [49]:
train.head()

,text,label
8061,Julie felt he was there,1
7455,I wonder has whether Mary worked for Microsoft.,0
7909,Aphrodite said he freed the animals and freed ...,0
5362,He offers more than we had hoped for.,1
1059,There were five tourists in the room apart fro...,1


In [50]:
from bert_sklearn import SUPPORTED_MODELS
SUPPORTED_MODELS

('bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese')

In [51]:
%%time
from sklearn.metrics import matthews_corrcoef

X_train = train['text']
y_train = train['label']

# define model
model = BertClassifier(bert_model='bert-base-multilingual-uncased')
model.epochs = 3
model.validation_fraction = 0.3
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 2

print('\n',model,'\n')

# fit model
model.fit(X_train, y_train)

# test model on dev
test = dev
X_test = test['text']
y_test = test['label']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

# Mathews correlation coefficient
print("\nMathews Correlation: %0.2f"%(matthews_corrcoef(y_test, y_pred) * 100))

Building sklearn classifier...

 BertClassifier(bert_model='bert-base-multilingual-uncased', epochs=3,
        eval_batch_size=8, fp16=False, gradient_accumulation_steps=2,
        label_list=None, learning_rate=2e-05, local_rank=-1,
        logfile='bert_sklearn.log', loss_scale=0, max_seq_length=128,
        num_mlp_hiddens=500, num_mlp_layers=0, random_state=42,
        restore_file=None, train_batch_size=32, use_cuda=True,
        validation_fraction=0.3, warmup_proportion=0.1) 



100%|██████████| 871891/871891 [00:00<00:00, 4695190.52B/s]


Loading bert-base-multilingual-uncased model...


100%|██████████| 623743758/623743758 [00:12<00:00, 49443557.05B/s]


Defaulting to linear classifier/regressor
train data size: 5986, validation data size: 2565


...

Epoch 2, Train loss : 0.6101, Val loss: 0.6076, Val accy = 70.21%


Training: 100%|██████████| 375/375 [04:57<00:00,  1.64it/s, loss=0.602]
                                                             

Epoch 3, Train loss : 0.6019, Val loss: 0.5985, Val accy = 70.21%


Accuracy: 69.13%
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       322
    positive       0.69      1.00      0.82       721

   micro avg       0.69      0.69      0.69      1043
   macro avg       0.35      0.50      0.41      1043
weighted avg       0.48      0.69      0.57      1043


Mathews Correlation: 0.00
CPU times: user 10min 23s, sys: 7min 1s, total: 17min 25s
Wall time: 17min 47s


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [53]:
%%time
from sklearn.metrics import matthews_corrcoef

X_train = train['text']
y_train = train['label']

# define model
model = BertClassifier(bert_model='bert-base-multilingual-cased')
model.epochs = 3
model.validation_fraction = 0.3
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 2

print('\n',model,'\n')

# fit model
model.fit(X_train, y_train)

# test model on dev
test = dev
X_test = test['text']
y_test = test['label']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

# Mathews correlation coefficient
print("\nMathews Correlation: %0.2f"%(matthews_corrcoef(y_test, y_pred) * 100))

Building sklearn classifier...

 BertClassifier(bert_model='bert-base-multilingual-cased', epochs=3,
        eval_batch_size=8, fp16=False, gradient_accumulation_steps=2,
        label_list=None, learning_rate=2e-05, local_rank=-1,
        logfile='bert_sklearn.log', loss_scale=0, max_seq_length=128,
        num_mlp_hiddens=500, num_mlp_layers=0, random_state=42,
        restore_file=None, train_batch_size=32, use_cuda=True,
        validation_fraction=0.3, warmup_proportion=0.1) 



100%|██████████| 995526/995526 [00:00<00:00, 2064663.69B/s]


Loading bert-base-multilingual-cased model...


100%|██████████| 662804195/662804195 [01:04<00:00, 10310949.41B/s]


Defaulting to linear classifier/regressor
train data size: 5986, validation data size: 2565


...

Epoch 2, Train loss : 0.6080, Val loss: 0.6062, Val accy = 70.45%


Training: 100%|██████████| 375/375 [04:59<00:00,  1.62it/s, loss=0.604]
                                                             

Epoch 3, Train loss : 0.6044, Val loss: 0.5986, Val accy = 70.45%


Accuracy: 69.13%
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       322
    positive       0.69      1.00      0.82       721

   micro avg       0.69      0.69      0.69      1043
   macro avg       0.35      0.50      0.41      1043
weighted avg       0.48      0.69      0.57      1043


Mathews Correlation: 0.00
CPU times: user 10min 29s, sys: 7min 5s, total: 17min 34s
Wall time: 18min 43s


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [57]:
%%time
from sklearn.model_selection import GridSearchCV

params = {'learning_rate':[2e-5, 3e-5], 'train_batch_size':[32, 64]}

# wrap classifier/regressor in GridSearchCV
clf = GridSearchCV(BertClassifier(epochs = 3), 
                    params,
                    scoring='accuracy',
                    verbose=True)

# fit gridsearch 
clf.fit(X_train ,y_train)

Building sklearn classifier...
Building sklearn classifier...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Building sklearn classifier...


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
100%|██████████| 231508/231508 [00:00<00:00, 2211325.02B/s]


Loading bert-base-uncased model...


100%|██████████| 407873900/407873900 [00:09<00:00, 42852060.87B/s]


Defaulting to linear classifier/regressor
train data size: 5130, validation data size: 570


...

Training: 100%|██████████| 161/161 [03:53<00:00,  1.22s/it, loss=0.319]
                                                           

Epoch 3, Train loss : 0.3191, Val loss: 0.4413, Val accy = 81.05%


Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 5130, validation data size: 570


Training:   0%|          | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


RuntimeError: ignored

In [0]:
# best scores
print(clf.best_score_, clf.best_params_)

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

In [0]:
%%time
from sklearn.metrics import matthews_corrcoef

X_train = train['text']
y_train = train['label']

# define model
model = BertClassifier()
model.epochs = 15
model.validation_fraction = 0.3
model.learning_rate = 2e-5
model.max_seq_length = 128
model.gradient_accumulation_steps = 2

print('\n',model,'\n')

# fit model
model.fit(X_train, y_train)

# test model on dev
test = dev
X_test = test['text']
y_test = test['label']

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred,y_test) * 100))
print(classification_report(y_test, y_pred, target_names=['negative','positive']))

# Mathews correlation coefficient
print("\nMathews Correlation: %0.2f"%(matthews_corrcoef(y_test, y_pred) * 100))

Building sklearn classifier...

 BertClassifier(bert_model='bert-base-uncased', epochs=15, eval_batch_size=8,
        fp16=False, gradient_accumulation_steps=2, label_list=None,
        learning_rate=2e-05, local_rank=-1, logfile='bert_sklearn.log',
        loss_scale=0, max_seq_length=128, num_mlp_hiddens=500,
        num_mlp_layers=0, random_state=42, restore_file=None,
        train_batch_size=32, use_cuda=True, validation_fraction=0.3,
        warmup_proportion=0.1) 



100%|██████████| 231508/231508 [00:00<00:00, 2643317.98B/s]


Loading bert-base-uncased model...


100%|██████████| 407873900/407873900 [00:09<00:00, 43842920.79B/s]


Defaulting to linear classifier/regressor
train data size: 5986, validation data size: 2565


Training:  26%|██▌       | 98/375 [01:19<03:48,  1.21it/s, loss=0.691]